# **Chapter 6 (Knot complement)**

## **cusp_meridian_finder**

- The *cusp_meridian_finder* functions finds the shortest curve on the cusp of the single-cusped manifold covers of $O^{333}_{2}$ $O^{333}_{3}$, or $O^{333}_{4}$.

- The input is a non-negative integer *indx* within the range of one of the lists *single_cusped* found in the previous chapter.

- Recall that for each of the $O^{333}_{2}$ $O^{333}_{3}$, and $O^{333}_{4}$ we have a different associated *single_cusped*

- Then we follow constructing the graph given by the tiling of the cusp as explained in chapter 6 (see Example 6.3).

  - The variable *graph* refers to the graph constructed by edges and vertices of the tiling and it is defined in a similar fashion to the graphs in Section 5.3 (see the previous notebook).
  
  - *face_graph* refers to the dual graph. 

In [ ]:
# Here we assume the array "single_cusped" from the previous section is pre-computed
# We need "defaultdict" for defining the graph
from collections import defaultdict
from copy import deepcopy
from collections import deque

def cusp_meridian_finder(indx):

    # Getting the permutation representations corresponding to x, y, z, w for a cover
    x = single_cusped[indx][0]
    y = single_cusped[indx][1]
    z = single_cusped[indx][2]
    w = single_cusped[indx][3]

    # Labeling the vertices and edges of a tile copy as in Figure 53
    e0 = "edge 0"
    e2 = "edge 2"
    v0 = "vertex 0"
    v1 = "vertex 1"
    v2 = "vertex 2"
    v5 = "vertex 5"

    # Defining the edges (edge classes)
    edges =[]
    visited = set()
    for i in range(24):
        edges.append(((e0, i ,+1), (e0, x[i], -1)))
        edges.append(((e2, i ,+1), (e2, z[i], -1)))

    # Defining the node classes
    node_classes = []
    for conjugacy_class in edges:
        start_nodes = set()
        end_nodes = set()
        for edge in conjugacy_class:
            edge_name, copy_number, edge_sign = edge
            if edge_name == e0 and edge_sign == 1:
                u, v = (v0, copy_number), (v1, copy_number)
            elif edge_name == e0 and edge_sign == -1:
                u, v = (v0, copy_number), (v2, copy_number)
            
            elif edge_name == e2 and edge_sign == 1:
                u, v = (v1, copy_number), (v5, copy_number)
            elif edge_name == e2 and edge_sign == -1:
                u, v = (v2, copy_number), (v5, copy_number)
            
            
            start_nodes.add(u)
            end_nodes.add(v)
        node_classes.append(start_nodes)
        node_classes.append(end_nodes)
    
    # This function is the same as the one used in Section 5.3 for merging classes with intersection
    def combine_sets_with_intersections(sets):
        merged = True  # Initialize a flag to control the merging loop
        
        while merged:
            merged = False  # Reset the flag for each pass
            for i in range(len(sets)):
                for j in range(i + 1, len(sets)):
                    if sets[i].intersection(sets[j]):  # Check for intersection
                        # Union the two sets with intersection
                        sets[i] = sets[i].union(sets[j])
                        sets.pop(j)  # Remove the j-th set
                        merged = True  # Set the flag to indicate a merge occurred
                        break  # Break inner loop to recheck from start
                if merged:
                    break  # Break outer loop to restart the merging check
        
        return sets


    node_classes = combine_sets_with_intersections(node_classes)
    node_classes = [tuple(node) for node in node_classes]


    # Defining the graph
    graph = defaultdict(list)

    for conjugacy_class in edges:
        edge_name, copy_number, edge_sign = conjugacy_class[0]
        if edge_name == e0 and edge_sign == 1:
            u, v = (v0, copy_number), (v1, copy_number)
        elif edge_name == e0 and edge_sign == -1:
            u, v = (v0, copy_number), (v2, copy_number)
            
        elif edge_name == e2 and edge_sign == 1:
            u, v = (v1, copy_number), (v5, copy_number)
        elif edge_name == e2 and edge_sign == -1:
            u, v = (v2, copy_number), (v5, copy_number)
        for node in node_classes:
            if u in node:
                start_node = node
                break
        for node in node_classes:
            if v in node:
                end_node = node
                break
        graph[start_node].append((end_node, {"edge": conjugacy_class, "direction": 1}))
        graph[end_node].append((start_node, {"edge": conjugacy_class, "direction": -1}))

    # Defining the dual graph
    faces_graph = defaultdict(set)
    for edge in edges:
        face1, face2 = edge[0][1], edge[1][1]
        faces_graph[face1].add(face2)
        faces_graph[face2].add(face1)
    
    # This function takes a cycle (as a list of edges) in the original graph and removes from the dual
    # In the sense that if two faces are connected in the dual through an edge which belongs to the cycle
    # We remove the connection
    def remove_cycle(cycle_edges):
        face_neibs = deepcopy(faces_graph)
        for edge in cycle_edges:
            
            face1, face2 = edge[0][1], edge[1][1]

            # removal of neighbors
            if face2 in face_neibs[face1]:
                face_neibs[face1].remove(face2)
            if face1 in face_neibs[face2]:
                face_neibs[face2].remove(face1)
                    
        return face_neibs
    
    # This function checks if a given cylce is a meridian
    # It removes the cycle from the dual
        # If the result is a disconnected graph, the cycle is not a meridian and function returns False
        # Otherwise we have a merdian and it returns True
    def is_cycle_meridian(cycle_edges):
        face_neibs = remove_cycle(cycle_edges)
        initial_face = cycle_edges[0][0][1]
        seen = set([initial_face])
        def dfs(node):
            for neib in face_neibs[node]:
                if neib not in seen:
                    seen.add(neib)
                    dfs(neib)
        dfs(initial_face)
        return len(seen) == 24
    
    # This function implements a BFS search to find the shortest cycle of edges satisfying:
        # Cycle has no repetitive edges (however it can have repetitive vertices as it can have multiple edges between a pair of vertices)
        # Cycle is a meridian

    def shortest_cycle_finder(graph, node):

        que = deque([([node], [])])
        while que:
            node_path, edge_path = que.popleft()
            for neib, edge_data in graph[node_path[-1]]:
                edge_path_edges = [edge_item['edge'] for edge_item in edge_path]
                if edge_data['edge'] not in edge_path_edges:
                    if neib in node_path:
                        cycle_edges = edge_path_edges + [edge_data['edge']]
                        if is_cycle_meridian(cycle_edges):
                            return edge_path + [edge_data]
                    elif neib not in node_path:
                        que.append((node_path + [neib], edge_path + [edge_data]))   
    meridian = shortest_cycle_finder(graph, node_classes[0])
    
    # Finally we express the meridian in terms of the edge labels of the original 2-complex as in Figure 51.
    meridian_edge_labels = []
    for edge_data in meridian:
        edge_name, copy_number, sign = edge_data['edge'][0]
        dir = edge_data['direction']
        if edge_name == 'edge 0' and sign == 1:
            meridian_edge_labels.append((('edge 2', copy_number, 1), dir))
        elif edge_name == 'edge 2' and sign == 1 and dir == 1:
            meridian_edge_labels.append((('edge 5', copy_number, 1), 1))
            meridian_edge_labels.append((('edge 7', copy_number, 1), -1))
        elif edge_name == 'edge 2' and sign == 1 and dir == -1:
            meridian_edge_labels.append((('edge 7', copy_number, 1), 1))
            meridian_edge_labels.append((('edge 5', copy_number, 1), -1))
    return meridian_edge_labels

## **Expressing the meridian in terms of the genrators of fundamental group of the cover**

- For each cover we rebuild the graph (define in the previous chapter) corresponding to the 1-skeleton of $X$ (as in **Lemma 5.11** and **Corollary 5.12**) corresponding to the cover.
- Then we use then we compare the edge labeling of the cycle to the generators found previously (edges outside the spanning tree).

## Relator corrsponding to a cusp meridian of a cover of $O^{333}_{2}$

- Here we assume that *single_cusped* is associated with $O^{333}_{2}$ and is pre-computed in the previous chapter.

- We can set *indx* to any non-negative integer within the range of the list *single_cusped* for $O^{333}_{2}$, i.e. $indx \in [0, 11]$.

- Then, we define the 1-skeleton of $X$ for the cover of $O^{333}_{2}$ that correponds to *single_cusped[indx]* just like the previous chapter.

- Finally, we compare the *meridian_edge_labels* obtained by *cusp_meridian_finder* with the generatos and produce the relator correposnding to the medridian in GAP language.

In [ ]:
# The NetWorkx library needs to be installed in your Python environment to run this function
import networkx as nx

# Here "single_cusped" is as in the previous chapter for O333-2
# i.e. it contains the generators of subgrpups corresponding to 24-fold single-cusped manifold covers of O333-2

indx = None                                             # you need to choose indx

sigma_x = single_cusped[indx][0]
sigma_y = single_cusped[indx][1]
sigma_z = single_cusped[indx][2]
sigma_w = single_cusped[indx][3]

# Defining Edge Classes as in Corollary 5.12(1)
e2 = "edge 2"
classes_e2 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_x[i]
        i_2 = sigma_y.index(i_1)
        i_3 = sigma_x[i_2]
        clas = [(e2, i, +1), (e2, i_1, -1), (e2, i_2, +1), (e2, i_3, -1)]
        classes_e2.append(clas)
        visited.update([i, i_2])
    else:
        continue
e3 = "edge 3"
classes_e3 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_y[i_1]
        clas = [(e3, i, +1), (e3, i_1, +1), (e3, i_2, +1)]
        classes_e3.append(clas)
        visited.update([i, i_1, i_2])
    else:
        continue
e5 = "edge 5"
classes_e5 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_z.index(i_1)
        i_3 = sigma_y[i_2]
        i_4 = sigma_z.index(i_3)
        i_5 = sigma_y[i_4]
        i_6 = sigma_z.index(i_5)
        i_7 = sigma_y[i_6]
        clas = [(e5, i, +1), (e5, i_1, -1), (e5, i_2, +1), (e5, i_3, -1), (e5, i_4, +1),(e5, i_5, -1), (e5, i_6, +1), (e5, i_7, -1)]
        classes_e5.append(clas)
        visited.update([i, i_2, i_4, i_6])
    else:
        continue
e6 = "edge 6"
classes_e6 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_w[i]
        clas = [(e6, i, +1), (e6, i_1, +1)]
        classes_e6.append(clas)
        visited.update([i, i_1])
    else:
        continue
e7 = "edge 7"
classes_e7 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_z[i]
        i_2 = sigma_w.index(i_1)
        i_3 = sigma_z[i_2]
        clas = [(e7, i, +1), (e7, i_1, -1), (e7, i_2, +1), (e7, i_3, -1)]
        classes_e7.append(clas)
        visited.update([i, i_2])
    else:
        continue
e8 = "edge 8"
classes_e8 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_w.index(i_1)
        i_3 = sigma_y[i_2]
        i_4 = sigma_w.index(i_3)
        i_5 = sigma_y[i_4]
        clas = [(e8, i, +1), (e8, i_1, -1), (e8, i_2, +1), (e8, i_3, -1), (e8, i_4, +1), (e8, i_5, -1)]
        classes_e8.append(clas)
        visited.update([i, i_2, i_4])
    else:
        continue
conjugacy_classes = classes_e2 + classes_e3 + classes_e5 + classes_e6 + classes_e7 + classes_e8

# Finding Vertex Classes
node_classes = []
for conjugacy_class in conjugacy_classes:
    start_nodes = set()
    end_nodes = set()
    for edge in conjugacy_class:
        edge_name, copy_number, edge_sign = edge
        if edge_name == 'edge 2' and edge_sign == 1:
            u, v = ('vertex 0', copy_number), ('vertex 1', copy_number)
        elif edge_name == 'edge 2' and edge_sign == -1:
            u, v = ('vertex 0', copy_number), ('vertex 2', copy_number)
        
        elif edge_name == 'edge 3' and edge_sign == 1:
            u, v = ('vertex 0', copy_number), ('vertex 3', copy_number)
        
        elif edge_name == 'edge 5' and edge_sign == 1:
            u, v = ('vertex 1', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 5' and edge_sign == -1:
            u, v = ('vertex 2', copy_number), ('vertex 6', copy_number)
        
        elif edge_name == 'edge 6' and edge_sign == 1:
            u, v = ('vertex 3', copy_number), ('vertex 5', copy_number)    
        elif edge_name == 'edge 7' and edge_sign == 1:
            u, v = ('vertex 5', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 7' and edge_sign == -1:
            u, v = ('vertex 5', copy_number), ('vertex 6', copy_number)
        elif edge_name == 'edge 8' and edge_sign == 1:
            u, v = ('vertex 3', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 8' and edge_sign == -1:
            u, v = ('vertex 3', copy_number), ('vertex 6', copy_number)
        start_nodes.add(u)
        end_nodes.add(v)
    node_classes.append(start_nodes)
    node_classes.append(end_nodes)
# This a standard function that given a collection of sets, merges those with intersection
def combine_sets_with_intersections(sets):
    merged = True  # Initialize a flag to control the merging loop
    
    while merged:
        merged = False  # Reset the flag for each pass
        for i in range(len(sets)):
            for j in range(i + 1, len(sets)):
                if sets[i].intersection(sets[j]):  # Check for intersection
                    # Union the two sets with intersection
                    sets[i] = sets[i].union(sets[j])
                    sets.pop(j)  # Remove the j-th set
                    merged = True  # Set the flag to indicate a merge occurred
                    break  # Break inner loop to recheck from start
            if merged:
                break  # Break outer loop to restart the merging check        
    return sets
node_classes = combine_sets_with_intersections(node_classes)
node_classes = [tuple(node) for node in node_classes]

# Initiate a graph and add the vertices (vertex classes) to it
G = nx.MultiGraph()
G.add_nodes_from(node_classes)

# Adding the edges (classes) to the graph
# The resulting graph is 1-skeleton of X

for conjugacy_class in conjugacy_classes:
    edge_name, copy_number, edge_sign = conjugacy_class[0]
    if edge_name == 'edge 2' and edge_sign == 1:
        u, v = ('vertex 0', copy_number), ('vertex 1', copy_number)
    elif edge_name == 'edge 2' and edge_sign == -1:
        u, v = ('vertex 0', copy_number), ('vertex 2', copy_number)
    
    elif edge_name == 'edge 3' and edge_sign == 1:
        u, v = ('vertex 0', copy_number), ('vertex 3', copy_number)
    
    elif edge_name == 'edge 5' and edge_sign == 1:
        u, v = ('vertex 1', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 5' and edge_sign == -1:
        u, v = ('vertex 2', copy_number), ('vertex 6', copy_number)
    
    elif edge_name == 'edge 6' and edge_sign == 1:
        u, v = ('vertex 3', copy_number), ('vertex 5', copy_number)
    elif edge_name == 'edge 7' and edge_sign == 1:
        u, v = ('vertex 5', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 7' and edge_sign == -1:
        u, v = ('vertex 5', copy_number), ('vertex 6', copy_number)
    elif edge_name == 'edge 8' and edge_sign == 1:
        u, v = ('vertex 3', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 8' and edge_sign == -1:
        u, v = ('vertex 3', copy_number), ('vertex 6', copy_number)
    for node_class in node_classes:
        if u in node_class:
            start_node = node_class
            break
    for node in node_classes:
        if v in node:
            end_node = node
            break
    idx = conjugacy_classes.index(conjugacy_class)
    G.add_edge(start_node, end_node, label=idx)

# Finding the spanning tree of the graph
# Setiting the edges (i.e. label of the edges classes) as the list of "generators"

spanning_tree = nx.minimum_spanning_tree(G)
generators = []
for edge in G.edges(data=True):
    if edge not in spanning_tree.edges(data=True):
        generators.append(edge[2]['label'])

# A map that sends the generators to strings 'x{i}'
# This makes it possible to feed this information to GAP
gen_map = {}
for i in range(len(generators)):
    gen_map[generators[i]] = f'x{i}'

# Using "gen_map" to translate generators to GAP language (in terms of 'x{i}')
# gap_gens ---> list of generators

gap_gens = [f"\"x{i}\"" for i in range(len(generators))]
gap_gens = ", ".join(gap_gens)



                                                                            # NOTE
                                                # Up to this point the graph buolding was the same as section 5.3
                                        # The next part is where we express the relator in terms of our generators in GAP language

meridian_edge_labels = cusp_meridian_finder(indx)

# A helper function
# Given an element edge in a an equivalency class of edges
# It finds the label of the class
def find_edge_label(edge):
        for edge_class in conjugacy_classes:
            if edge in edge_class:
                return conjugacy_classes.index(edge_class)
            
labels = [(find_edge_label(a), b) for a, b in meridian_edge_labels]

# This is the relator in terms of edge labels
relator_labels = []
for x in labels:
    if x[0] in generators:
        relator_labels.append(x)

# This is the relator in terms of the generators (named as x{i})
relator_word = []
for (label, sign) in relator_labels:
    char = gen_map[label]
    if sign == 1:
        relator_word.append(char)
    else:
        relator_word.append(char + "^-1")

# This is the relator in GAP language 
# Concistent with the way we defined the fundamental group previously

product = " * ".join([f"f.{int(var[1:]) + 1}" if '^' not in var else f"f.{int(var[1:].split('^')[0]) + 1}^{var.split('^')[1]}" for var in relator_word])
relator = f"    {product}"
print(relator)


## **Files for $O^{333}_{2}$**

- We have found the *relator* for the single-cusped manifold covers of $O^{333}_{2}$ with $\mathbb{Z}$ homology. From previous chapter we know that those are:
    - *single_cusped[i]* for *i* in the list *[2, 4, 6, 8, 10, 11]*

- We added this relator to the presentation of the fundamental group of the cover.

- Next we utilized GAP's **SimplifiedFpGroup** command which uses Tietze transformations to simplify the presentation of the group.

- Then we used GAP's **IsTrivial** function to check whether we get a trivial group after adding the relator (i.e. After Dehn filling).

- The GAP scripts containing the definition of the group with the extra relator, and the command *IsTrivial* are stored in files under the following names:
     - **gap_dehn_filled_O333_2_i.g** for every i in the list *[2, 4, 6, 8, 10, 11]*

- The results are recorded in **Theorem 6.1.**


## Relator corrsponding to a cusp meridian of a cover of $O^{333}_{3}$

- Here we assume that *single_cusped* is associated with $O^{333}_{3}$ and is pre-computed in the previous chapter.

- We can set *indx* to any non-negative integer within the range of the list *single_cusped* for $O^{333}_{3}$, i.e. $indx \in [0, 11]$.

- Then, we define the 1-skeleton of $X$ for the cover of $O^{333}_{3}$ that correponds to *single_cusped[indx]* just like the previous chapter.

- Finally, we compare the *meridian_edge_labels* obtained by *cusp_meridian_finder* with the generatos and produce the relator correposnding to the medridian in GAP language.

In [ ]:
# The NetWorkx library needs to be installed in your Python environment to run this function
import networkx as nx

# Here "single_cusped" is as in the previous chapter for O333-2
# i.e. it contains the generators of subgrpups corresponding to 24-fold single-cusped manifold covers of O333-2

indx = None                                  # you need to choose indx

sigma_x = single_cusped[indx][0]
sigma_y = single_cusped[indx][1]
sigma_z = single_cusped[indx][2]
sigma_w = single_cusped[indx][3]

# Defining Edge Classes as in Corollary 5.12(2)
e2 = "edge 2"
classes_e2 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_x[i]
        i_2 = sigma_y.index(i_1)
        i_3 = sigma_x[i_2]
        clas = [(e2, i, +1), (e2, i_1, -1), (e2, i_2, +1), (e2, i_3, -1)]
        classes_e2.append(clas)
        visited.update([i, i_2])
    else:
        continue
e3 = "edge 3"
classes_e3 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_y[i_1]
        clas = [(e3, i, +1), (e3, i_1, +1), (e3, i_2, +1)]
        classes_e3.append(clas)
        visited.update([i, i_1, i_2])
    else:
        continue
e5 = "edge 5"
classes_e5 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_z.index(i_1)
        i_3 = sigma_y[i_2]
        i_4 = sigma_z.index(i_3)
        i_5 = sigma_y[i_4]
        i_6 = sigma_z.index(i_5)
        i_7 = sigma_y[i_6]
        clas = [(e5, i, +1), (e5, i_1, -1), (e5, i_2, +1), (e5, i_3, -1), (e5, i_4, +1),(e5, i_5, -1), (e5, i_6, +1), (e5, i_7, -1)]
        classes_e5.append(clas)
        visited.update([i, i_2, i_4, i_6])
    else:
        continue
e6 = "edge 6"
classes_e6 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_w[i]
        clas = [(e6, i, +1), (e6, i_1, +1)]
        classes_e6.append(clas)
        visited.update([i, i_1])
    else:
        continue
e7 = "edge 7"
classes_e7 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_z[i]
        i_2 = sigma_w.index(i_1)
        i_3 = sigma_z[i_2]
        i_4 = sigma_w.index(i_3)
        i_5 = sigma_z[i_4]
        clas = [(e7, i, +1), (e7, i_1, -1), (e7, i_2, +1), (e7, i_3, -1), (e7, i_4, +1), (e7, i_5, -1)]
        classes_e7.append(clas)
        visited.update([i, i_2, i_4])
    else:
        continue
e8 = "edge 8"
classes_e8 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_w.index(i_1)
        i_3 = sigma_y[i_2]
        clas = [(e8, i, +1), (e8, i_1, -1), (e8, i_2, +1), (e8, i_3, -1)]
        classes_e8.append(clas)
        visited.update([i, i_2])
    else:
        continue
conjugacy_classes = classes_e2 + classes_e3 + classes_e5 + classes_e6 + classes_e7 + classes_e8

# Finding Vertex Classes
node_classes = []
for conjugacy_class in conjugacy_classes:
    start_nodes = set()
    end_nodes = set()
    for edge in conjugacy_class:
        edge_name, copy_number, edge_sign = edge
        if edge_name == 'edge 2' and edge_sign == 1:
            u, v = ('vertex 0', copy_number), ('vertex 1', copy_number)
        elif edge_name == 'edge 2' and edge_sign == -1:
            u, v = ('vertex 0', copy_number), ('vertex 2', copy_number)
        
        elif edge_name == 'edge 3' and edge_sign == 1:
            u, v = ('vertex 0', copy_number), ('vertex 3', copy_number)
        
        elif edge_name == 'edge 5' and edge_sign == 1:
            u, v = ('vertex 1', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 5' and edge_sign == -1:
            u, v = ('vertex 2', copy_number), ('vertex 6', copy_number)
        
        elif edge_name == 'edge 6' and edge_sign == 1:
            u, v = ('vertex 3', copy_number), ('vertex 5', copy_number)    
        elif edge_name == 'edge 7' and edge_sign == 1:
            u, v = ('vertex 5', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 7' and edge_sign == -1:
            u, v = ('vertex 5', copy_number), ('vertex 6', copy_number)
        elif edge_name == 'edge 8' and edge_sign == 1:
            u, v = ('vertex 3', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 8' and edge_sign == -1:
            u, v = ('vertex 3', copy_number), ('vertex 6', copy_number)
        start_nodes.add(u)
        end_nodes.add(v)
    node_classes.append(start_nodes)
    node_classes.append(end_nodes)
# This a standard function that given a collection of sets, merges those with intersection
def combine_sets_with_intersections(sets):
    merged = True  # Initialize a flag to control the merging loop
    
    while merged:
        merged = False  # Reset the flag for each pass
        for i in range(len(sets)):
            for j in range(i + 1, len(sets)):
                if sets[i].intersection(sets[j]):  # Check for intersection
                    # Union the two sets with intersection
                    sets[i] = sets[i].union(sets[j])
                    sets.pop(j)  # Remove the j-th set
                    merged = True  # Set the flag to indicate a merge occurred
                    break  # Break inner loop to recheck from start
            if merged:
                break  # Break outer loop to restart the merging check        
    return sets
node_classes = combine_sets_with_intersections(node_classes)
node_classes = [tuple(node) for node in node_classes]

# Initiate a graph and add the vertices (vertex classes) to it
G = nx.MultiGraph()
G.add_nodes_from(node_classes)

# Adding the edges (classes) to the graph
# The resulting graph is 1-skeleton of X

for conjugacy_class in conjugacy_classes:
    edge_name, copy_number, edge_sign = conjugacy_class[0]
    if edge_name == 'edge 2' and edge_sign == 1:
        u, v = ('vertex 0', copy_number), ('vertex 1', copy_number)
    elif edge_name == 'edge 2' and edge_sign == -1:
        u, v = ('vertex 0', copy_number), ('vertex 2', copy_number)
    
    elif edge_name == 'edge 3' and edge_sign == 1:
        u, v = ('vertex 0', copy_number), ('vertex 3', copy_number)
    
    elif edge_name == 'edge 5' and edge_sign == 1:
        u, v = ('vertex 1', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 5' and edge_sign == -1:
        u, v = ('vertex 2', copy_number), ('vertex 6', copy_number)
    
    elif edge_name == 'edge 6' and edge_sign == 1:
        u, v = ('vertex 3', copy_number), ('vertex 5', copy_number)
    elif edge_name == 'edge 7' and edge_sign == 1:
        u, v = ('vertex 5', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 7' and edge_sign == -1:
        u, v = ('vertex 5', copy_number), ('vertex 6', copy_number)
    elif edge_name == 'edge 8' and edge_sign == 1:
        u, v = ('vertex 3', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 8' and edge_sign == -1:
        u, v = ('vertex 3', copy_number), ('vertex 6', copy_number)
    for node_class in node_classes:
        if u in node_class:
            start_node = node_class
            break
    for node in node_classes:
        if v in node:
            end_node = node
            break
    idx = conjugacy_classes.index(conjugacy_class)
    G.add_edge(start_node, end_node, label=idx)

# Finding the spanning tree of the graph
# Setiting the edges (i.e. label of the edges classes) as the list of "generators"

spanning_tree = nx.minimum_spanning_tree(G)
generators = []
for edge in G.edges(data=True):
    if edge not in spanning_tree.edges(data=True):
        generators.append(edge[2]['label'])

# A map that sends the generators to strings 'x{i}'
# This makes it possible to feed this information to GAP
gen_map = {}
for i in range(len(generators)):
    gen_map[generators[i]] = f'x{i}'

# Using "gen_map" to translate generators to GAP language (in terms of 'x{i}')
# gap_gens ---> list of generators

gap_gens = [f"\"x{i}\"" for i in range(len(generators))]
gap_gens = ", ".join(gap_gens)



                                                                            # NOTE
                                                # Up to this point the graph buolding was the same as section 5.3
                                        # The next part is where we express the relator in terms of our generators in GAP language

meridian_edge_labels = cusp_meridian_finder(indx)

# A helper function
# Given an element edge in a an equivalency class of edges
# It finds the label of the class
def find_edge_label(edge):
        for edge_class in conjugacy_classes:
            if edge in edge_class:
                return conjugacy_classes.index(edge_class)
            
labels = [(find_edge_label(a), b) for a, b in meridian_edge_labels]

# This is the relator in terms of edge labels
relator_labels = []
for x in labels:
    if x[0] in generators:
        relator_labels.append(x)

# This is the relator in terms of the generators (named as x{i})
relator_word = []
for (label, sign) in relator_labels:
    char = gen_map[label]
    if sign == 1:
        relator_word.append(char)
    else:
        relator_word.append(char + "^-1")

# This is the relator in GAP language 
# Concistent with the way we defined the fundamental group previously

product = " * ".join([f"f.{int(var[1:]) + 1}" if '^' not in var else f"f.{int(var[1:].split('^')[0]) + 1}^{var.split('^')[1]}" for var in relator_word])
relator = f"    {product}"
print(relator)


## **Files for $O^{333}_{3}$**

- We have found the *relator* for the single-cusped manifold covers of $O^{333}_{3}$ with $\mathbb{Z}$ homology. From previous chapter we know that those are:
    - *single_cusped[i]* for *i* in the list *[2, 5, 6, 7, 8, 9]*

- We added this relator to the presentation of the fundamental group of the cover.

- Next we utilized GAP's **SimplifiedFpGroup** command which uses Tietze transformations to simplify the presentation of the group.

- Then we used GAP's **IsTrivial** function to check whether we get a trivial group after adding the relator (i.e. After Dehn filling).

- The GAP scripts containing the definition of the group with the extra relator, and the command *IsTrivial* are stored in files under the following names:
     - **gap_dehn_filled_O333_3_i.g** for every i in the list *[2, 5, 6, 7, 8, 9]*

- The results are recorded in **Theorem 6.1.**


## Relator corrsponding to a cusp meridian of a cover of $O^{333}_{4}$

- Here we assume that *single_cusped* is associated with $O^{333}_{4}$ and is pre-computed in the previous chapter.

- We can set *indx* to any non-negative integer within the range of the list *single_cusped* for $O^{333}_{4}$, i.e. $indx \in [0, 62]$.

- Then, we define the 1-skeleton of $X$ for the cover of $O^{333}_{4}$ that correponds to *single_cusped[indx]* just like the previous chapter.

- Finally, we compare the *meridian_edge_labels* obtained by *cusp_meridian_finder* with the generatos and produce the relator correposnding to the medridian in GAP language.

In [ ]:
# The NetWorkx library needs to be installed in your Python environment to run this function
import networkx as nx

# Here "single_cusped" is as in the previous chapter for O333-2
# i.e. it contains the generators of subgrpups corresponding to 24-fold single-cusped manifold covers of O333-2

indx = None                                  # you need to choose indx

sigma_x = single_cusped[indx][0]
sigma_y = single_cusped[indx][1]
sigma_z = single_cusped[indx][2]
sigma_w = single_cusped[indx][3]

# Defining Edge Classes as in Corollary 5.12(3)
e2 = "edge 2"
classes_e2 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_x[i]
        i_2 = sigma_y.index(i_1)
        i_3 = sigma_x[i_2]
        clas = [(e2, i, +1), (e2, i_1, -1), (e2, i_2, +1), (e2, i_3, -1)]
        classes_e2.append(clas)
        visited.update([i, i_2])
    else:
        continue
e3 = "edge 3"
classes_e3 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_y[i_1]
        clas = [(e3, i, +1), (e3, i_1, +1), (e3, i_2, +1)]
        classes_e3.append(clas)
        visited.update([i, i_1, i_2])
    else:
        continue
e5 = "edge 5"
classes_e5 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_z.index(i_1)
        i_3 = sigma_y[i_2]
        i_4 = sigma_z.index(i_3)
        i_5 = sigma_y[i_4]
        i_6 = sigma_z.index(i_5)
        i_7 = sigma_y[i_6]
        clas = [(e5, i, +1), (e5, i_1, -1), (e5, i_2, +1), (e5, i_3, -1), (e5, i_4, +1),(e5, i_5, -1), (e5, i_6, +1), (e5, i_7, -1)]
        classes_e5.append(clas)
        visited.update([i, i_2, i_4, i_6])
    else:
        continue
e6 = "edge 6"
classes_e6 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_w[i]
        i_2 = sigma_w[i_1]
        clas = [(e6, i, +1), (e6, i_1, +1), (e6, i_2, +1)]
        classes_e6.append(clas)
        visited.update([i, i_1, i_2])
    else:
        continue
e7 = "edge 7"
classes_e7 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_z[i]
        i_2 = sigma_w.index(i_1)
        i_3 = sigma_z[i_2]
        clas = [(e7, i, +1), (e7, i_1, -1), (e7, i_2, +1), (e7, i_3, -1)]
        classes_e7.append(clas)
        visited.update([i, i_2, i_4])
    else:
        continue
e8 = "edge 8"
classes_e8 = []
visited = set()
for i in range(24):
    if i not in visited:
        i_1 = sigma_y[i]
        i_2 = sigma_w.index(i_1)
        i_3 = sigma_y[i_2]
        clas = [(e8, i, +1), (e8, i_1, -1), (e8, i_2, +1), (e8, i_3, -1)]
        classes_e8.append(clas)
        visited.update([i, i_2])
    else:
        continue
conjugacy_classes = classes_e2 + classes_e3 + classes_e5 + classes_e6 + classes_e7 + classes_e8

# Finding Vertex Classes
node_classes = []
for conjugacy_class in conjugacy_classes:
    start_nodes = set()
    end_nodes = set()
    for edge in conjugacy_class:
        edge_name, copy_number, edge_sign = edge
        if edge_name == 'edge 2' and edge_sign == 1:
            u, v = ('vertex 0', copy_number), ('vertex 1', copy_number)
        elif edge_name == 'edge 2' and edge_sign == -1:
            u, v = ('vertex 0', copy_number), ('vertex 2', copy_number)
        
        elif edge_name == 'edge 3' and edge_sign == 1:
            u, v = ('vertex 0', copy_number), ('vertex 3', copy_number)
        
        elif edge_name == 'edge 5' and edge_sign == 1:
            u, v = ('vertex 1', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 5' and edge_sign == -1:
            u, v = ('vertex 2', copy_number), ('vertex 6', copy_number)
        
        elif edge_name == 'edge 6' and edge_sign == 1:
            u, v = ('vertex 3', copy_number), ('vertex 5', copy_number)    
        elif edge_name == 'edge 7' and edge_sign == 1:
            u, v = ('vertex 5', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 7' and edge_sign == -1:
            u, v = ('vertex 5', copy_number), ('vertex 6', copy_number)
        elif edge_name == 'edge 8' and edge_sign == 1:
            u, v = ('vertex 3', copy_number), ('vertex 4', copy_number)
        elif edge_name == 'edge 8' and edge_sign == -1:
            u, v = ('vertex 3', copy_number), ('vertex 6', copy_number)
        start_nodes.add(u)
        end_nodes.add(v)
    node_classes.append(start_nodes)
    node_classes.append(end_nodes)
# This a standard function that given a collection of sets, merges those with intersection
def combine_sets_with_intersections(sets):
    merged = True  # Initialize a flag to control the merging loop
    
    while merged:
        merged = False  # Reset the flag for each pass
        for i in range(len(sets)):
            for j in range(i + 1, len(sets)):
                if sets[i].intersection(sets[j]):  # Check for intersection
                    # Union the two sets with intersection
                    sets[i] = sets[i].union(sets[j])
                    sets.pop(j)  # Remove the j-th set
                    merged = True  # Set the flag to indicate a merge occurred
                    break  # Break inner loop to recheck from start
            if merged:
                break  # Break outer loop to restart the merging check        
    return sets
node_classes = combine_sets_with_intersections(node_classes)
node_classes = [tuple(node) for node in node_classes]

# Initiate a graph and add the vertices (vertex classes) to it
G = nx.MultiGraph()
G.add_nodes_from(node_classes)

# Adding the edges (classes) to the graph
# The resulting graph is 1-skeleton of X

for conjugacy_class in conjugacy_classes:
    edge_name, copy_number, edge_sign = conjugacy_class[0]
    if edge_name == 'edge 2' and edge_sign == 1:
        u, v = ('vertex 0', copy_number), ('vertex 1', copy_number)
    elif edge_name == 'edge 2' and edge_sign == -1:
        u, v = ('vertex 0', copy_number), ('vertex 2', copy_number)
    
    elif edge_name == 'edge 3' and edge_sign == 1:
        u, v = ('vertex 0', copy_number), ('vertex 3', copy_number)
    
    elif edge_name == 'edge 5' and edge_sign == 1:
        u, v = ('vertex 1', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 5' and edge_sign == -1:
        u, v = ('vertex 2', copy_number), ('vertex 6', copy_number)
    
    elif edge_name == 'edge 6' and edge_sign == 1:
        u, v = ('vertex 3', copy_number), ('vertex 5', copy_number)
    elif edge_name == 'edge 7' and edge_sign == 1:
        u, v = ('vertex 5', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 7' and edge_sign == -1:
        u, v = ('vertex 5', copy_number), ('vertex 6', copy_number)
    elif edge_name == 'edge 8' and edge_sign == 1:
        u, v = ('vertex 3', copy_number), ('vertex 4', copy_number)
    elif edge_name == 'edge 8' and edge_sign == -1:
        u, v = ('vertex 3', copy_number), ('vertex 6', copy_number)
    for node_class in node_classes:
        if u in node_class:
            start_node = node_class
            break
    for node in node_classes:
        if v in node:
            end_node = node
            break
    idx = conjugacy_classes.index(conjugacy_class)
    G.add_edge(start_node, end_node, label=idx)

# Finding the spanning tree of the graph
# Setiting the edges (i.e. label of the edges classes) as the list of "generators"

spanning_tree = nx.minimum_spanning_tree(G)
generators = []
for edge in G.edges(data=True):
    if edge not in spanning_tree.edges(data=True):
        generators.append(edge[2]['label'])

# A map that sends the generators to strings 'x{i}'
# This makes it possible to feed this information to GAP
gen_map = {}
for i in range(len(generators)):
    gen_map[generators[i]] = f'x{i}'

# Using "gen_map" to translate generators to GAP language (in terms of 'x{i}')
# gap_gens ---> list of generators

gap_gens = [f"\"x{i}\"" for i in range(len(generators))]
gap_gens = ", ".join(gap_gens)



                                                                            # NOTE
                                                # Up to this point the graph buolding was the same as section 5.3
                                        # The next part is where we express the relator in terms of our generators in GAP language

meridian_edge_labels = cusp_meridian_finder(indx)

# A helper function
# Given an element edge in a an equivalency class of edges
# It finds the label of the class
def find_edge_label(edge):
        for edge_class in conjugacy_classes:
            if edge in edge_class:
                return conjugacy_classes.index(edge_class)
            
labels = [(find_edge_label(a), b) for a, b in meridian_edge_labels]

# This is the relator in terms of edge labels
relator_labels = []
for x in labels:
    if x[0] in generators:
        relator_labels.append(x)

# This is the relator in terms of the generators (named as x{i})
relator_word = []
for (label, sign) in relator_labels:
    char = gen_map[label]
    if sign == 1:
        relator_word.append(char)
    else:
        relator_word.append(char + "^-1")

# This is the relator in GAP language 
# Concistent with the way we defined the fundamental group previously

product = " * ".join([f"f.{int(var[1:]) + 1}" if '^' not in var else f"f.{int(var[1:].split('^')[0]) + 1}^{var.split('^')[1]}" for var in relator_word])
relator = f"    {product}"
print(relator)


## **Files for $O^{333}_{4}$**

- We have found the *relator* for the single-cusped manifold covers of $O^{333}_{4}$ with $\mathbb{Z}$ homology. From previous chapter we know that those are:
    - *single_cusped[i]* for *i* in the list *[9, 10, 14, 16, 20, 25, 31, 32, 53, 60]*

- We added this relator to the presentation of the fundamental group of the cover.

- Next we utilized GAP's **SimplifiedFpGroup** command which uses Tietze transformations to simplify the presentation of the group.

- Then we used GAP's **IsTrivial** function to check whether we get a trivial group after adding the relator (i.e. After Dehn filling).

- The GAP scripts containing the definition of the group with the extra relator, and the command *IsTrivial* are stored in files under the following names:
     - **gap_dehn_filled_O333-4-i.g** for every i in the list *[9, 10, 14, 16, 20, 25, 31, 32, 53, 60]*

- The results are recorded in **Theorem 6.1.**
